In [ ]:

%matplotlib widget

# Prototype Development
> Development Log of 9 Prototypes for demonstrating the different components of the semantic abstraction process

In [1]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
# #| export

# import matplotlib.pyplot as plt
# import numpy as np
# import open3d as o3d
# import copy
# import random

## Overview

The paper describes a method for creating abstracted geometry based on an initial pre-segmented input model. This is transformed into an abstracted version by applying Gestalt principles on objects according to sketches by the user. Groupings of the scene elements according to these rules are computed and abstractions are created summarizing the objects into bounding volumes or replacing geometries with scaled versions, depending on the intent of the user abstraction

### Applying Gestalt principles to 3D geometry

Determining potentials for Abstraction in the scene, based on the Gestalt principles is the most Interesting part of this method.


* Generate scene including a set of clear patterns (grid, dense cluster, line, tight grid, isolated cubes)
* Compute groups based on Gestalt principles (proximity, regularity, continuity, symmetry)


## Implementation

In this notebook I implement a demonstration of the core principles of using gestalt rules to guiding the abstraction of a scene.

For this we load / generate a simple scene with primitives in it ordered and aligned in a way so that different patterns and Gestalt shapes emerge naturally to the viewer. 

Based on this the program implements metrics from the paper to programmatically figure out these structures and mark the objects accordingly.

The program then visualizes this intial guess and lets the user show his intent of which of the objects to abstract.

Now using the intent and the previous knowledge a simple abstraction is computed and also rendered in the scene

This interactive loop now should already provide a basic understanding of how the method from the paper works albeit being very simple in its design.


### Determining Orientation and Shape Dimensions using PCA



In [ ]:
# # Set random seed for reproducibility
# random.seed(42)
# np.random.seed(42)

In [ ]:
# # | export
# # Generate a simple scene with cube primitives in structured arrangement
# def generate_cube_grid(n_x=4, n_y=3, spacing=1.5):
#     cube = o3d.geometry.TriangleMesh.create_box(width=1.0, height=1.0, depth=1.0)
#     cube.compute_vertex_normals()
#     scene = []
#     for i in range(n_x):
#         for j in range(n_y):
#             new_cube = copy.deepcopy(cube)
#             new_cube.translate(np.array([i * spacing, j * spacing, 0]))
#             scene.append(new_cube)
#     return scene

In [ ]:
# # | export
# def generate_custom_scene():
#     cube = o3d.geometry.TriangleMesh.create_box(width=1.0, height=1.0, depth=1.0)
#     cube.compute_vertex_normals()
#     scene = []

#     def place_grid(start, nx, ny, dx=1.5, dy=1.5):
#         for i in range(nx):
#             for j in range(ny):
#                 c = copy.deepcopy(cube)
#                 c.translate([start[0] + i * dx, start[1] + j * dy, start[2]])
#                 scene.append(c)

#     def place_line(start, count, d=1.5):
#         for i in range(count):
#             c = copy.deepcopy(cube)
#             c.translate([start[0] + i * d, start[1], start[2]])
#             scene.append(c)

#     def place_cluster(center, count=6, spread=0.6):
#         for _ in range(count):
#             c = copy.deepcopy(cube)
#             offset = np.random.normal(scale=spread, size=3)
#             c.translate(center + offset)
#             scene.append(c)

#     # Pattern 1: Structured Grid → suggests regularity and proximity
#     place_grid(start=np.array([0, 0, 0]), nx=3, ny=2)

#     # Pattern 2: Line → suggests continuity
#     place_line(start=np.array([7, 0, 0]), count=5)

#     # Pattern 3: Dense Cluster → suggests proximity, but low regularity
#     place_cluster(center=np.array([0, 6, 0]), count=7)

#     # Pattern 4: Small tight grid → proximity + potential symmetry
#     place_grid(start=np.array([7, 6, 0]), nx=2, ny=2, dx=1.0, dy=1.0)

#     # Pattern 5: Short isolated line → separate group
#     place_line(start=np.array([3, 3, 0]), count=3)

#     return scene

In [ ]:
# # | export
# # Create axis-aligned bounding box for a group of cubes
# # color them based on their regularity scores
# def create_group_aabb(group, regularity_score):
#     all_points = np.vstack([np.asarray(cube.get_axis_aligned_bounding_box().get_box_points()) for cube in group])
#     aabb = o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(all_points))

#     # Color code based on score
#     if regularity_score >= 0.8:
#         aabb.color = (0.0, 1.0, 0.0)  # Green = regular
#     elif regularity_score >= 0.5:
#         aabb.color = (1.0, 0.65, 0.0)  # Orange = moderate
#     else:
#         aabb.color = (1.0, 0.0, 0.0)  # Red = irregular
#     return aabb


### Proximity Metric

The proximity metric groups objects that are close to one another into a single group

In [ ]:
# # | export
# # Grouping by proximity (Euclidean distance between centers)
# def group_by_proximity(cubes, threshold=2.0):
#     centers = [cube.get_center() for cube in cubes]
#     groups = []
#     used = set()
#     for i in range(len(cubes)):
#         if i in used:
#             continue
#         group = [cubes[i]]
#         used.add(i)
#         for j in range(i + 1, len(cubes)):
#             if j not in used and np.linalg.norm(centers[i] - centers[j]) < threshold:
#                 group.append(cubes[j])
#                 used.add(j)
#         groups.append(group)
#     return groups

### Regularity Scoring
As a second metric in combination with proximity i include  a regularity score for each group.

Color bounding boxes based on that score:
* 🟢 green = high regularity (≥ 0.8)
* 🟠 orange = medium regularity (0.5–0.8)
* 🔴 red = low regularity (< 0.5)


In [ ]:
# # | export
# def compute_group_regularity(group):
#     if len(group) < 3:
#         return 0.0  # too small to assess regularity

#     centers = np.array([cube.get_center() for cube in group])
#     diffs = []

#     for i in range(len(centers)):
#         for j in range(i + 1, len(centers)):
#             diff = centers[j] - centers[i]
#             if np.linalg.norm(diff) > 1e-6:
#                 diffs.append(diff)

#     if len(diffs) < 3:
#         return 0.0  # not enough meaningful direction vectors

#     diffs = np.array(diffs)
#     norm_diffs = np.linalg.norm(diffs, axis=1, keepdims=True)
#     unit_dirs = diffs / norm_diffs

#     # PCA via covariance matrix of unit directions
#     try:
#         cov = np.cov(unit_dirs.T)
#         eigenvalues, _ = np.linalg.eigh(cov)
#         principal_val = eigenvalues[-1]  # largest eigenvalue = dominant direction
#         return float(np.clip(principal_val, 0.0, 1.0))  # clip to avoid NaNs
#     except np.linalg.LinAlgError:
#         return 0.0


In [ ]:
# Run demo
# scene = generate_custom_scene()
# groups = group_by_proximity(scene, threshold=2.2)
#
# bounding_boxes = []
# for i, group in enumerate(groups):
#     reg = compute_group_regularity(group)
#     print(f"Group {i}: Size={len(group)}, Regularity={reg:.2f}")
#     aabb = create_group_aabb(group, reg)
#     bounding_boxes.append(aabb)


Group 0: Size=4, Regularity=0.60
Group 1: Size=2, Regularity=0.00
Group 2: Size=2, Regularity=0.00
Group 3: Size=2, Regularity=0.00
Group 4: Size=1, Regularity=0.00
Group 5: Size=7, Regularity=0.46
Group 6: Size=4, Regularity=0.60
Group 7: Size=2, Regularity=0.00
Group 8: Size=1, Regularity=0.00


In [ ]:

# Visualize
#o3d.visualization.draw_geometries(scene + bounding_boxes)

In [ ]:
# from abc import ABC, abstractmethod
# from typing import Tuple


# class Evaluator(ABC):
#     """
#     Any grouping method must override `evaluate`.
#     It should return:
#       - groups:   list[list[o3d.TriangleMesh]]
#       - scores:   list[float]  (one per group, 0-1, or None if not meaningful)
#     """
#     name: str            # short label used in menus
#     color: Tuple[float]  # default RGB for this method’s boxes

#     @abstractmethod
#     def evaluate(self, cubes) -> tuple[list, list]:
#         ...


# class ProximityEval(Evaluator):
#     name, color = "Proximity", (0.2, 0.6, 1.0)  # light-blue

#     def evaluate(self, cubes):
#         groups = group_by_proximity(cubes, threshold=2.2)
#         # No per-group quality score → fill with None
#         return groups, [None]*len(groups)


# class RegularityEval(Evaluator):
#     name, color = "Regularity", (0.0, 0.8, 0.2)  # green

#     def evaluate(self, cubes):
#         groups = group_by_proximity(cubes, threshold=2.2)  # reuse
#         scores = [compute_group_regularity(g) for g in groups]
#         return groups, scores


# def groups_to_bboxes(groups, scores, base_color):
#     boxes = []
#     for g, s in zip(groups, scores):
#         aabb = create_group_aabb(g, s if s is not None else 1.0)
#         # over-write color so each evaluator keeps its own palette
#         aabb.color = base_color
#         boxes.append(aabb)
#     return boxes


# evaluators = [ProximityEval(), RegularityEval()]
# state = {"idx": 0, "scene": scene}  # mutable dict so the lambda sees updates

# def refresh(vis):
#     vis.clear_geometries()
#     vis.add_geometry(*state["scene"])             # raw cubes
#     ev = evaluators[state["idx"]]
#     groups, scores = ev.evaluate(state["scene"])
#     for box in groups_to_bboxes(groups, scores, ev.color):
#         vis.add_geometry(box)
#     print(f"Showing: {ev.name}")
#     return False                                  # tell Open3D to redraw

# def next_method(vis):
#     state["idx"] = (state["idx"] + 1) % len(evaluators)
#     return refresh(vis)

# key_to_callback = {ord("N"): next_method}  # press ‘N’ to cycle
# o3d.visualization.draw_geometries_with_key_callbacks(state["scene"], key_to_callback,
#                                                      window_name="Gestalt inspector")


TypeError: add_geometry(): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.cpu.pybind.visualization.Visualizer, geometry: open3d.cpu.pybind.geometry.Geometry, reset_bounding_box: bool = True) -> bool

Invoked with: VisualizerWithKeyCallback with name Gestalt inspector, TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 triangles., TriangleMesh with 8 points and 12 tr

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()